In [1]:
import torch
from transformers import AutoTokenizer,AutoModelForTokenClassification

c:\Users\imars\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import pandas as pd

data = pd.read_csv('data_with_entity.csv')

v_data = [' '.join(data['sent'].sample().to_list()) for i in range(100)]

In [10]:
def convert_from_torch_to_onnx(
        onnx_path: str,
        tokenizer: AutoTokenizer,
        model: AutoModelForTokenClassification,
        text: list
) -> None:
    """Конвертация модели из формата PyTorch в формат ONNX.

    @param onnx_path: путь к модели в формате ONNX
    @param tokenizer: токенизатор
    @param model: модель
    """
    dummy_model_input = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to("cpu")
    torch.onnx.export(
        model,
        dummy_model_input["input_ids"],
        onnx_path,
        opset_version=12,
        input_names=["input_ids"],
        output_names=["output"],
        dynamic_axes={
            "input_ids": {
                0: "batch_size",
                1: "sequence_len"
            },
            "output": {
                0: "batch_size"
            }
        }
    )

In [11]:
model_name = 'ai-forever/ruElectra-small'
label2idx = {'O': 0, 'B-discount': 1, 'B-value': 2, 'I-value': 3}
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels = len(label2idx)).to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\imars\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at ai-forever/ruElectra-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
convert_from_torch_to_onnx('ruElectra-small-onnx.onnx', tokenizer, model, v_data)

In [4]:
from onnxruntime.quantization import (
    quantize_dynamic,
    QuantType
)


def convert_from_onnx_to_quantized_onnx(
        onnx_model_path: str,
        quantized_onnx_model_path: str
) -> None:
    """Квантизация модели в формате ONNX до Int8
    и сохранение кванитизированной модели на диск.

    @param onnx_model_path: путь к модели в формате ONNX
    @param quantized_onnx_model_path: путь к квантизированной модели
    """
    quantize_dynamic(
        onnx_model_path,
        quantized_onnx_model_path,
        weight_type=QuantType.QUInt8
    )

In [14]:
convert_from_onnx_to_quantized_onnx('ruElectra-small-onnx.onnx', 'ruElectra-small-onnx-quantized.onnx')

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)


torch.set_num_threads(1)


def pytorch_inference(
        text: str,
        max_tokens: int,
        model: AutoModelForSequenceClassification,
        tokenizer: AutoTokenizer,
) -> torch.Tensor:
    """Инференс модели с помощью PyTorch.

    @param text: входной текст для классификации
    @param max_tokens: максимальная длина последовательности в токенах
    @param model: BERT-модель
    @param tokenizer: токенизатор
    @return: логиты на выходе из модели
    """
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_tokens,
        return_tensors="pt"
    ).to('cpu')
    with torch.inference_mode():
        outputs = model(**inputs).logits.detach()
    return outputs

In [ ]:
import onnxruntime
from onnxruntime import (
    InferenceSession,
    SessionOptions
)


def create_onnx_session(
        model_path: str,
        provider: str = "CPUExecutionProvider"
) -> InferenceSession:
    """Создание сессии для инференса модели с помощью ONNX Runtime.

    @param model_path: путь к модели в формате ONNX
    @param provider: инференс на ЦП
    @return: ONNX Runtime-сессия
    """    
    options = SessionOptions()
    options.graph_optimization_level = \
        onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    options.intra_op_num_threads = 1
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    return session

In [ ]:
import numpy as np
from transformers import AutoTokenizer
from onnxruntime import InferenceSession


def onnx_inference(
        text: str,
        session: InferenceSession,
        tokenizer: AutoTokenizer,
        max_length: int
) -> np.ndarray:
    """Инференс модели с помощью ONNX Runtime.

    @param text: входной текст для классификации
    @param session: ONNX Runtime-сессия
    @param tokenizer: токенизатор
    @param max_length: максимальная длина последовательности в токенах
    @return: логиты на выходе из модели
    """
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="np",
    )
    input_feed = {
        "input_ids": inputs["input_ids"].astype(np.int64)
    }
    outputs = session.run(
        output_names=["output"],
        input_feed=input_feed
    )[0]
    return outputs